# Ubuntu Dialogue - Visualization

This notebook is an analog of anssel-visual.ipynb for the Ubuntu dataset, which requires slight tweaks due to the fact that original texts are not loaded.  But it's actually a bit of an advantage that you easily see which words are out-of-vocabulary (OOV) in the listing.

At any rate, this notebook is much rawer.  Please refer to anssel-visual.ipynb for a variety of comments.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import print_function
from __future__ import division

import importlib
import numpy as np
try:
    import cPickle
except ImportError:  # python3
    import pickle as cPickle
import pickle
import random
import sys

from keras.callbacks import ModelCheckpoint
from keras.layers.core import Activation, Dropout
from keras.layers.recurrent import SimpleRNN, GRU, LSTM
from keras.models import Graph
from keras.preprocessing.sequence import pad_sequences

import pysts.embedding as emb
import pysts.eval as ev
import pysts.loader as loader
import pysts.nlp as nlp
from pysts.hyperparam import hash_params
from pysts.vocab import Vocabulary

from pysts.kerasts import graph_input_anssel
import pysts.kerasts.blocks as B
from pysts.kerasts.callbacks import AnsSelCB
from pysts.kerasts.objectives import ranknet, ranksvm, cicerons_1504

import anssel_train

Using Theano backend.
Using gpu device 0: Tesla K20m (CNMeM is disabled)


In [3]:
s0pad = 160
s1pad = 160


In [4]:
from ubuntu_train import *

In [ ]:
model = None

In [7]:
modelname, vocabf, trainf, valf = ('attn1511', '../data/anssel/ubuntu/v1-vocab.pickle',
                                   '../data/anssel/ubuntu/v1-trainset.pickle', '../data/anssel/ubuntu/v1-valset.pickle')

In [6]:
params = []

In [8]:
    module = importlib.import_module('.'+modelname, 'models')
    conf, ps, h = config(module.config, params)

In [9]:
    runid = '%s-%x' % (modelname, h)
    print('RunID: %s  (%s)' % (runid, ps))
    
    print('GloVe')
    glove = emb.GloVe(N=conf['embdim'])

RunID: attn1511-7d2b273301a5c213  ({"Ddim": "2", "adim": "0.5", "attn_mode": "sum", "batch_size": "192", "cdim": "2", "cfiltlen": "3", "cnnact": "tanh", "cnninit": "glorot_uniform", "dropout": "0.75", "e_add_flags": "True", "embdim": "300", "focus_act": "softmax", "inp_e_dropout": "0.75", "l2reg": "0.0001", "loss": "<function ranknet at 0x1086c488>", "mlpsum": "sum", "pool_layer": "<class 'keras.layers.convolutional.MaxPooling1D'>", "project": "True", "ptscorer": "<function mlp_ptscorer at 0x10872668>", "rnn": "<class 'keras.layers.recurrent.GRU'>", "rnnact": "tanh", "rnnbidi": "True", "rnnbidi_mode": "sum", "rnninit": "glorot_uniform", "rnnlevels": "1", "sdim": "2"})
GloVe


In [10]:
    print('Dataset (vocab)')
    vocab = pickle.load(open(vocabf, "rb"))  # use plain pickle because unicode

Dataset (vocab)


In [ ]:
    print('Dataset (train)')
    gr = load_set(trainf, vocab)

In [11]:
    print('Dataset (val)')
    grt = load_set(valf, vocab)
    

Dataset (val)


In [12]:
    print('Padding (val)')
    pad_graph(grt)

Padding (val)


In [13]:
model = anssel_train.build_model(glove, vocab, module.prep_model, conf, s0pad=s0pad, s1pad=s1pad)

pysts/vocab.py:56: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  embedding_weights[index, :] = emb.g[word]


In [14]:
model.load_weights('../ubu-weights-attn1511-687b949de0fe9cbe-bestval.h5')

In [16]:
model.count_params() - model.nodes['emb'].count_params()

6475658

In [17]:
ypredt = model.predict(grt)['score'][:,0]

In [18]:
ypredt

array([ 4.88575554, -0.45427024, -0.26975641, ..., -1.46304238,
       -0.40381491, -1.74145603])

In [19]:
ev.eval_ubuntu(ypredt, grt['si0'], grt['score'], 'Val')

Val MRR: 0.593857
Val 2-R@1: 0.761887
Val 10-R@1: 0.444123  10-R@2: 0.565763  10-R@5: 0.792529


(0.59385674608747219,
 0.76188737536862805,
 0.44412301643027663,
 0.5657632355006319,
 0.79252913916584744)

## Attention Visualization

In [21]:
from pysts.kerasts import graph_input_slice
sl = slice(500, 1000)
grs = graph_input_slice(grt, sl)

In [22]:
import theano
def layer_fun(model, gr, layer_name):
    thf = theano.function([model.inputs[name].input for name in model.input_order],
                          model.nodes[layer_name].get_output(train=False),
                          on_unused_input='ignore', allow_input_downcast=True)
    return thf(*[gr[name] for name in model.input_order])

def predict_internal(model, gr):
    pred = model.predict(gr)
    ypred = pred['score'][:, 0]
    if 'tokens' in pred:
        tpred = pred['tokens']
    else:
        tpred = None

    # e1a is e0-driven attention
    # (use e1a[3] inst. of e1a[2] to get the softmax focus)
    e1a = layer_fun(model, gr, 'e1a[2]')
    # e0c, e1c are convolutions that are max-pooled for summary embedding
    # (so more important areas should get higher convolution norm?)
    e0c = layer_fun(model, gr, 'e0c')
    e1c = layer_fun(model, gr, 'e1c')
    return (ypred, tpred, e1a, e0c, e1c)

ypred, tpred, e1a, e0c, e1c = predict_internal(model, grs)

In [26]:
idx2word = {v: k for k, v in vocab.word_idx.items()}

In [27]:
def predict_table(idx2word, gr, e0rgb, e1rgb):
    from IPython.display import HTML
    from numpy.linalg import norm
    h = []
    s0 = [[idx2word[i] for i in s if i != 0] for s in gr['si0']]
    s1 = [[idx2word[i] for i in s if i != 0] for s in gr['si1']]
    for i in range(len(s0)):
        def rgbnorm(rgb, i):
            rgbi = [0, 0, 0]
            rgbin = [0, 0, 0]
            for j in range(3):
                try:
                    rgbi[j] = [norm(e) for e in rgb[j][i]]
                    rgbin[j] = rgbi[j] / np.max(rgbi[j])
                except TypeError:  # 0 inst. of list
                    rgbi[j] = [0 for e in range(anssel_train.s0pad)]
                    rgbin[j] = rgbi[j]
            return (rgbi, rgbin)
        e0rgbi, e0rgbin = rgbnorm(e0rgb, i)
        e1rgbi, e1rgbin = rgbnorm(e1rgb, i)

        def tokcolor(rgb, rgbn, j, t):
            return ('</span><span style="background: rgb(%d,%d,%d)" title="%.3f | %.3f | %.3f">%s' %
                    (128+rgbn[0][j]*128, 128+rgbn[1][j]*128, 128+rgbn[2][j]*128,
                     norm(rgb[0][j]), norm(rgb[1][j]), norm(rgb[2][j]), t))
        toks0 = ' '.join([tokcolor(e0rgbi, e0rgbin, j, t) for j, t in enumerate(s0[i][:38])])
        toks1 = ' '.join([tokcolor(e1rgbi, e1rgbin, j, t) for j, t in enumerate(s1[i][:38])])

        h.append('<tr style="%s"><td style="color: rgb(%d,0,0)">%.3f<td>%d<td>%s<td>%s' %
                 (' font-weight: bold' if gr['score'][i] == 1. else '',
                  0, ypred[i], gr['score'][i],
                  toks0, toks1))
    return HTML('<table>' + ''.join(h) + '</table>')

In [28]:
predict_table(idx2word, grs, [e0c, 0, 0], [e1c, 0, e1a])